# Cables Characterization

In [1]:
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np
import os
import sys

sys.path.insert(1, '..\\Instruments\\network\\RS_Signal_Generator')
sys.path.insert(2, '..\\Instruments\\Acquisition_Card')
sys.path.insert(4, '..\\HDF5')

import RS_SMA100B
from Card_Class import PXIe5170R
from HDF5 import *

In [3]:
SG_ip           = '192.168.40.15'                                   # Set IP address of the device
Card_Name       = 'PXI1Slot3'

sGen            = RS_SMA100B.SMA100B(ip = SG_ip)
pxie            = PXIe5170R(resource_name = Card_Name)

sGen.reset()
sGen.clear()

SMA100B:	Connection successful!
PXIe_5170R:    Connection successful!


In [5]:
sample_rate     = 250e6                                             # Maximum Value: 250.0e6
k = 4                                                               # coefficiente per prendere k*1000 punti 
																	# con la scheda, dunque devo aumentare la pulse width e period
pulse_period    = k * 1e-6                                          # 4e-6 con 250e6 dà 1000 punti
num_pts         = int(sample_rate * pulse_period)                   # min 5ns                             
pulse_delay     = 0
percent         = 5
pulse_width     = pulse_period * (1 - percent/100)                  # min 20ns


# parameters for the Signal Generator
sGen.pul_gen_params(delay = pulse_delay, 
                    width = pulse_width, 
                    period = pulse_period)   # da capire quale pulse width
sGen.pul_gen_mode('SING')
sGen.pul_trig_mode('SING')


# parameters for the Acquisition Card
pxie.voltage_range      = 0.5
pxie.coupling           = 'AC' # or DC????
pxie.num_pts            = num_pts
pxie.num_records        = 1
pxie.ref_position       = 0
pxie.sample_rate        = sample_rate                               # Maximum Value: 250.0e6

In [10]:
# -------------------------- #

cable_name      = 'C_A'
pulse_freq      = 1.010e9
amp_min		    = -20                                               # Set the amplitude of the signal in dBm
amp_max         = 20
amps            = np.arange(amp_min, amp_max, 1)

# -------------------------- #

trig_CH     = 0
cable_CH    = 1

sGen.RF_freq(pulse_freq) 
pxie.open(trigger_channel = str(trig_CH))

data_dict   = {cable_name: {'freq_(Hz)': pulse_freq, 'powers': {}}}
counter     = 1

digits_f = "{:0"+str(len(str(len(amps))))+"d}"


for i, amp in enumerate(amps):
	
	sGen.RF_lvl_ampl(amp)
	sGen.pul_state(1)
	sGen.RF_state(1)
	waveforms = pxie.acquisition(sGen.pul_exe_sing_trig)
	sweep = np.array(waveforms[cable_CH].samples.tolist())
	
	sGen.pul_state(0)
	sGen.RF_state(0)

	print(f'\rf{digits_f.format(i)}	: {int(counter*100/len(amps))} %', end='')
	sys.stdout.flush()
	counter += 1
	data_dict['powers'][f'f{digits_f.format(i)}'] = {'power_(dBm)': amp, 'sweep': sweep}


f39	: 100 %

In [11]:
# SAVE DATA ON HDF5 FILE


filename = 'cables.h5'
# if os.path.exists(filename):
#   os.remove(filename)
save_dict_to_hdf5(data_dict, filename)